# UltraInsight Project Code
## Description


### Setup

In [ ]:
#setup
import numpy as np # linear algebra
import pandas as pd # data processing

#insert and read file
filename = "../archive (2)/TWO_CENTURIES_OF_UM_RACES.csv"
df = pd.read_csv(filename)
print(df)
df.head()

### Data Preprocessing and Cleaning

In [ ]:
# select most recent 10 years
year_cutoff = df['Year of event'].max() - 10
print(len(df))

df = df.dropna()
df = df.drop_duplicates()
print(len(df))

year_inds = df['Year of event'] >= year_cutoff
df = df[year_inds]
print(len(df))

print(df["Event distance/length"])

In [ ]:
#select km and mi races only, not timed ones
distance = df["Event distance/length"]
non_time_km = distance.str.contains('km', regex=False)
non_time_mi = distance.str.endswith('mi')

#setup 2 different dataframes, one with km and one with mi races
df1 = df[non_time_km]
df2 = df[non_time_mi]
print(df1)
print(df2)
print(len(df1))
print(len(df2))

#Take the mi, mile, and miles races and strip them of their ending letters
conversion_kit = df2["Event distance/length"]
print(conversion_kit)
miles = conversion_kit.str.rstrip('miles')
miles = miles.str.rstrip('mile')
miles = miles.str.rstrip('mi')
print(miles)

#manual cleaning up of commas to convert to string floats
miles = miles.str.replace('37,5','37.5')
miles = miles.str.replace('52,5','52.5')
miles = miles.str.replace('52,4','52.4')
miles = miles.str.replace('67,5','67.5')

#convert mi to km
mi_to_km = 1.60934 * miles.astype(float)
print('In kms')
print(mi_to_km)


In [ ]:
#remove km ending from km df
conversion = df1["Event distance/length"]
#conversion = conversion.rename(columns = {"Event distance/length" : "Event distance"})
#print(conversion)
km = conversion

# non_time_km = distance.str.contains('km', regex=False)
good_inds = ~km.str.contains('.Etappen', regex=True) & \
    ~km.str.contains('.Etap', regex=True) & \
    ~km.str.contains(' Sp', regex=False) & \
    ~km.str.contains('.x.', regex=True) & \
    ~km.str.contains('.stag', regex=True)
print(len(good_inds))
print(len(df1))
# km = km[~km.str.contains('.Etappen', regex=True)]
# km = km[~km.str.contains('.Etap', regex=True)]
# km = km[~km.str.contains(' Sp', regex=False)]
# km = km[~km.str.contains('.x.', regex=True)]
# km = km[~km.str.contains('.stag', regex=True)]
km = km[good_inds]

km = km.str.rstrip('kilometers')
km = km.str.rstrip('kilometer')
km = km.str.rstrip('km')
km = km.str.rstrip('km ')

# km to float
km = km.astype(float)
print(km.mean())
print(mi_to_km.mean())
df1 = df1[good_inds]

# replace column with numeric values
# print(len(df1))
df1['Event distance/length'] = km
df2['Event distance/length'] = mi_to_km
print(df1['Event distance/length'].mean())
print(df2['Event distance/length'].mean())

giga_chad_df = pd.concat([df1, df2])
print(giga_chad_df['Event distance/length'].mean())
giga_chad_df = giga_chad_df.rename(columns = {"Event distance/length" : "Event distance"})
df = giga_chad_df
#combine both dfs
# merged_km = [mi_to_km, km]
# # distance = pd.concat(merged_km)
# #distance = distance.rename(columns = {"Event distance/length" : "Event distance"})
# print(distance.mean())

# #reintegrate back into main dataframe
# df["Event distance/length"].update(distance)
#df["Event distance/length"] = df["Event distance/length"].method(distance)
# print(df)
# acceptable_dist_inds = distance.index
# print(acceptable_dist_inds)
# print(distance)
# df = df[acceptable_dist_inds]
# print(df["Event distance/length"])
# print(df.index.max())
# print(acceptable_dist_inds.max())
# df[df.index.isin(acceptable_dist_inds)]


In [ ]:
#rename age category
df = df.rename(columns = {"Athlete age category" : "Athlete age"})

#remove letters from in front of ages (W,MU,M)
rm_age_cat = df["Athlete age"]
print(rm_age_cat)
age = rm_age_cat.str.lstrip('MU')
age = age.str.lstrip('M')
age = age.str.lstrip('W')
age = age.str.lstrip('F')
age = age.str.lstrip('U')
print(age)
age = age.astype(int)

#reintegrate back into main dataframe
df["Athlete age"].update(age)
print(df)
print(df["Athlete age"].mean())


### Data Analysis

In [ ]:
#calculate pace

df3 = df["Event distance"]

df4 = df["Athlete performance"]
df4 = df4.str.replace(':','.')
df4 = df4.str.replace(' ','0')

good_inds = ~df4.str.contains('d', regex=True)
df4 = df4[good_inds]
df3 = df3[good_inds]

df["Event distance"] = df3
df["Athlete performance"] = df4
    # ~km.str.contains('.Etap', regex=True) & \
    # ~km.str.contains(' Sp', regex=False) & \
    # ~km.str.contains('.x.', regex=True) & \
    # ~km.str.contains('.stag', regex=True)
# df4 = df4[:-4]

# merged_data1 = [df3, df4]
# pace_df = pd.concat(merged_data1)
# # print(pace_df['Athlete performance'].mean())
# df = df["Athlete performance"][good_inds]

hr = df4.str.rstrip('hour')
hr = hr.str.rstrip('hr')
hr = hr.str.rstrip('h')
hr = hr.str.replace('.', '|', regex = False)
hr = hr.str.split(pat='|', expand=True)
hr = hr.astype(int)
time_in_mins = 60.0*hr[0] + hr[1] + hr[2]/60.0
print(time_in_mins)

pace = pd.to_numeric(df3)/ time_in_mins
print(pace)
df.insert(loc=len(df.columns), column='Pace(km/min)', value=pace)

# print(df)
# temp_df = pd.concat([df3, df4])
# print(temp_df['Athlete performance'].mean())
# df = temp_df
# print(df)

In [ ]:
# pd.Series(['a|b', 'a', 'a|c']).str.get_dummies()
#    a  b  c
# 0  1  1  0
# 1  1  0  0
# 2  1  0  1

In [ ]:
#average pace per race distance

#50km
for i in len(df["Event distance"]):
   if df["Event distance"] >= 50 & df["Event distance"] < 51:
      avg_pace_50km = df["Event distance"] / time_in_mins
print(avg_pace_50km)

#80km
for i in len(df["Event distance"]):
   if df["Event distance"] >= 80 & df["Event distance"] < 81:
      avg_pace_80km = df["Event distance"] / time_in_mins
print(avg_pace_80km)

#100km
for i in len(df["Event distance"]):
   if df["Event distance"] >= 100 & df["Event distance"] < 101:
      avg_pace_100km = df["Event distance"] / time_in_mins
print(avg_pace_100km)

#161km
for i in len(df["Event distance"]):
   if df["Event distance"] >= 160 & df["Event distance"] < 161:
      avg_pace_161km = df["Event distance"] / time_in_mins
print(avg_pace_161km)

In [ ]:
#correlation of top 20% in field per race

In [ ]:
#stdev for outliers of pace per race distance